# Installation

In [ ]:
!pip install plyvel

In [ ]:
!pip install rlp

In [ ]:
!pip install eth

In [ ]:
!pip install eth-rlp

In [ ]:
!pip install eth-rlp eth-utils rlp

In [ ]:
!pip install eth_account

In [ ]:
!pip install web3

In [ ]:
import rlp
from eth_utils import to_hex, to_int
from rlp.sedes import big_endian_int, binary
import plyvel
from eth_account._utils.signing import extract_chain_id, to_standard_v
from eth_keys import keys
from eth_utils import to_checksum_address
from eth_utils import decode_hex
from web3 import Web3
from web3.auto import w3
import re

#Code

In [ ]:
!unzip '/content/node_infos.zip'

Archive:  /content/node_infos.zip
   creating: node_infos/
  inflating: node_infos/000001.log   
 extracting: node_infos/CURRENT      
 extracting: node_infos/LOCK         
  inflating: node_infos/LOG          
 extracting: node_infos/MANIFEST-000000  


#Only Read

In [ ]:
import plyvel

def verificar_leveldb(caminho_db):
    try:
        # Abrindo o banco de dados em modo de leitura
        db = plyvel.DB(caminho_db, create_if_missing=False)
        print("Banco de dados aberto com sucesso.")

        # Percorrendo todas as chaves e valores
        for chave, valor in db:
            print(f"Chave: {chave}, Valor: {valor}")

        # Fechando o banco de dados
        db.close()
    except Exception as e:
        print(f"Erro ao abrir/verificar o banco de dados: {str(e)}")

# Exemplo de uso
caminho = "/content/leveldb_data_2"
verificar_leveldb(caminho)

Banco de dados aberto com sucesso.
Chave: b'DatabaseVersion', Valor: b'\x08'
Chave: b'H\x06\x19\x96\x01\xc9\xbf\xb3)\xba\xef\x90!\x809<\xd1(\xa4+q\xb1>\xa1j\xd9+{\xed\xd0\xa5q\xa3', Valor: b'\x00\x00\x00\x00\x00\x00\x00\x1c'
Chave: b'H\x14\xa2\xc57_\x8d\xd0\x93X\x00\x84\x8f\x04c\xdc\xa7\xba\xde\xe5\xcc\x96\x8aL(e\x1b\xb6}\x0eS%{', Valor: b'\x00\x00\x00\x00\x00\x00\x00\x17'
Chave: b'H\x15\xbd\xc5\x84\xb7:Wj\x1b?\x02\xd0\x1f\xc4V\x111\xe7\xfc\x92\xc0\x1a\xbe\xd2\xee"\xacfO\t\xfam', Valor: b'\x00\x00\x00\x00\x00\x00\x00 '
Chave: b'H\x17\xca\x94\xb1\xb2\xc1$\xac\xb2\xfa\x99\xa6m\xcc\x19H\x82w\x9d\x16h\xd1fzn\xf8\x9c\xed\xd3v\xe7q', Valor: b'\x00\x00\x00\x00\x00\x00\x00\x14'
Chave: b'H\x18_\x1e\xb6\x11h>\xcd\x16\xd8\x834\xec\x01g\xf0QX\xae\xa8\xdf>\xef\xf4\x810\xeb\x9fj\xd5M\xe3', Valor: b'\x00\x00\x00\x00\x00\x00\x00\x0f'
Chave: b'H=\x7f\x8bQ\xbfV\xb1\xc5a\x15\x066q\x92\xd5P!nQTD\xc8\xbfn{\xa3Nz\x8d\xf5\xf0\xa2', Valor: b'\x00\x00\x00\x00\x00\x00\x00\x1a'
Chave: b'H=\xe9\xf4\xa3-y\xbd\xf5\

#Clean Read

In [ ]:
def calcular_hash_transacao(tx_fields):
    """Recalcula o hash da transação serializada em RLP."""
    try:
        tx_serializado = rlp.encode(tx_fields)
        tx_hash = Web3.keccak(tx_serializado)
        return tx_hash.hex()
    except Exception as e:
        print(f"Erro ao calcular hash da transação: {str(e)}")
        return None

def extrair_from(tx_fields):
    """Extrai o remetente ('from') usando a recuperação da assinatura."""
    try:
        # Constrói a transação RLP sem os campos de assinatura (v, r, s)
        tx_unsigned = rlp.encode(tx_fields[:-3])  # Exclui v, r e s
        tx_hash = Web3.keccak(tx_unsigned)

        # Extrai os componentes da assinatura
        v = int.from_bytes(tx_fields[-3], byteorder='big')
        r = int.from_bytes(tx_fields[-2], byteorder='big')
        s = int.from_bytes(tx_fields[-1], byteorder='big')

        # Realiza a recuperação do endereço do remetente
        sender = w3.eth.account.recover_transaction(
            rlp.encode(tx_fields)
        )
        return to_checksum_address(sender)
    except Exception as e:
        print(f"Erro ao extrair 'from': {str(e)}")
        return None

def verificar_leveldb(caminho_db):
  try:
      db = plyvel.DB(caminho_db, create_if_missing=False)
      print("Banco de dados aberto com sucesso.\n")
      print(db)
      for chave, valor in db:
          decoded_string_data = valor.decode(errors='ignore')
          matches = re.findall(r"'checklist'", decoded_string_data, re.DOTALL)

          # Processa cada match_content apenas uma vez
          for match_content in matches:
              try:
                  # Decodifica o valor uma vez para cada match_content
                  decoded_data = rlp.decode(valor)

                  # Processa todas as transações em decoded_data[0]
                  for tx_fields in decoded_data[0]:
                      # Extrai campos principais
                      nonce = int.from_bytes(tx_fields[0], byteorder='big')
                      gas_price = int.from_bytes(tx_fields[1], byteorder='big')
                      gas_limit = int.from_bytes(tx_fields[2], byteorder='big')

                      # Verifica o endereço de destino ou se é um contrato
                      to_address = tx_fields[3]
                      if to_address != b'':
                          to_address = to_checksum_address(to_address.hex())
                      else:
                          to_address = "Contrato Criado"

                      value = int.from_bytes(tx_fields[4], byteorder='big')
                      data = tx_fields[5].hex() if len(tx_fields) > 5 else "Sem dados"

                      # Extrai o endereço do remetente (from)
                      from_address = extrair_from(tx_fields)

                      # Recalcula o hash da transação
                      tx_hash = calcular_hash_transacao(tx_fields)

                      # Decodifica possíveis dados em formato JSON-like
                      decoded_string_data = valor.decode(errors='ignore')
                      match_content = re.findall(r"\{.*?\}", decoded_string_data, re.DOTALL)

                      # Exibe os resultados
                      print(' ')
                      print(f"Hash da Transação: 0x{tx_hash}")
                      print(f"From: {from_address}")
                      print(f"Nonce: {nonce}")
                      print(f"To: {to_address}")
                      print(f"Value: {value}")
                      print(f"Data (hex): {data}")
                      print(f"Conteúdo Encontrado: {match_content}")
                      print("-" * 50)

                  # Sai do loop após processar uma vez
                  break

              except Exception as e:
                  print(f"Erro ao decodificar valor: {str(e)}")
                  continue

      db.close()

  except Exception as e:
      print(f"Erro ao abrir/verificar o banco de dados: {str(e)}")

# Exemplo de uso
caminho = "/content/node_infos"
verificar_leveldb(caminho)

Banco de dados aberto com sucesso.

<plyvel.DB with name '/content/node_infos' at 0x0x789fa9595bc0>
 
Hash da Transação: 0xa6a1a0c35a154edeffd2e949bdfedbc1a4d57b7083de1ba73faeca056d296897
From: 0xA231229C933A93D70B6cE3d58e159Af91074E58D
Nonce: 0
To: Contrato Criado
Value: 0
Data (hex): 60806040526040518060400160405280601081526020017f4e6f20526567697374726174696f6e21000000000000000000000000000000008152506000908051906020019061004f929190610062565b5034801561005c57600080fd5b50610166565b82805461006e90610105565b90600052602060002090601f01602090048101928261009057600085556100d7565b82601f106100a957805160ff19168380011785556100d7565b828001600101855582156100d7579182015b828111156100d65782518255916020019190600101906100bb565b5b5090506100e491906100e8565b5090565b5b808211156101015760008160009055506001016100e9565b5090565b6000600282049050600182168061011d57607f821691505b6020821081141561013157610130610137565b5b50919050565b7f4e487b710000000000000000000000000000000000000000000000000000000060005260226004526024600